# 生成train.txt 和 test.txt文件

## 该部分函数由yolo3_keras的voc_annotation.py函数修改而来。因为需要依据我们自身的数据文件特性进行针对性修改。

In [45]:
import os
import random
import xml.etree.ElementTree as ET

from utils.utils import get_classes

#--------------------------------------------------------------------------------------------------------------------------------#
#   annotation_mode用于指定该文件运行时计算的内容
#   annotation_mode为0代表整个标签处理过程，包括获得./ImageSets里面的txt以及训练用的train.txt、val.txt
#   annotation_mode为1代表获得./ImageSets里面的txt
#   annotation_mode为2代表获得训练用的train.txt、val.txt
#--------------------------------------------------------------------------------------------------------------------------------#
annotation_mode     = 0
#-------------------------------------------------------------------#
#   必须要修改，用于生成train.txt、val.txt的目标信息
#   与训练和预测所用的classes_path一致即可
#   如果生成的train.txt里面没有目标信息
#   那么就是因为classes没有设定正确
#   仅在annotation_mode为0和2的时候有效
#-------------------------------------------------------------------#
classes_path        = './model_data/labels.txt'
#--------------------------------------------------------------------------------------------------------------------------------#
#   trainval_percent用于指定(训练集+验证集)与测试集的比例，默认情况下 (训练集+验证集):测试集 = 9:1
#   train_percent用于指定(训练集+验证集)中训练集与验证集的比例，默认情况下 训练集:验证集 = 9:1
#   仅在annotation_mode为0和1的时候有效

# 
#--------------------------------------------------------------------------------------------------------------------------------#
#trainval_percent    = 0.9
#train_percent       = 0.9
trainval_percent    = 0.9
train_percent       = 0.9
#-------------------------------------------------------#
#   指向数据集所在的文件夹
#   默认指向根目录下的VOC数据集
#-------------------------------------------------------#
VOCdevkit_path  = 'E:/university/2022 semester'

image_classes = ["001.Black_footed_Albatross",
"002.Laysan_Albatross",
"003.Sooty_Albatross",
"004.Groove_billed_Ani",
"005.Crested_Auklet",
"006.Least_Auklet",
"007.Parakeet_Auklet",
"008.Rhinoceros_Auklet",
"009.Brewer_Blackbird",
"010.Red_winged_Blackbird"]

#例如：image_id = "Black_Footed_Albatross_0001_796111"

#VOCdevkit_sets  = [('2007', 'train'), ('2007', 'val')]
classes, _      = get_classes(classes_path)

def convert_annotation(image_class,image_id, list_file):
    """
    image_classes:文件名称（在这里文件名和类别名一致）
    image_id:某一类中的文件的id名
    list_file:用于保存的目的文件
    """
    in_file = open(os.path.join(VOCdevkit_path, 'data_bird_box/annotations/train/%s/%s.xml'%(image_class,image_id)), encoding='utf-8')
    tree=ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        difficult = 0 
        if obj.find('difficult')!=None:
            difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(float(xmlbox.find('xmin').text)), int(float(xmlbox.find('ymin').text)), int(float(xmlbox.find('xmax').text)), int(float(xmlbox.find('ymax').text)))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))

"""
if __name__ == "__main__":
    random.seed(0)
    if annotation_mode == 0 or annotation_mode == 1:
        print("Generate txt in ImageSets.")
        xmlfilepath     = os.path.join(VOCdevkit_path, 'VOC2007/Annotations')
        saveBasePath    = os.path.join(VOCdevkit_path, 'VOC2007/ImageSets/Main')
        temp_xml        = os.listdir(xmlfilepath)
        total_xml       = []
        for xml in temp_xml:
            if xml.endswith(".xml"):
                total_xml.append(xml)

        num     = len(total_xml)  
        list    = range(num)  
        tv      = int(num*trainval_percent)  
        tr      = int(tv*train_percent)  
        trainval= random.sample(list,tv)  
        train   = random.sample(trainval,tr)  
        
        print("train and val size",tv)
        print("train size",tr)
        ftrainval   = open(os.path.join(saveBasePath,'trainval.txt'), 'w')  
        ftest       = open(os.path.join(saveBasePath,'test.txt'), 'w')  
        ftrain      = open(os.path.join(saveBasePath,'train.txt'), 'w')  
        fval        = open(os.path.join(saveBasePath,'val.txt'), 'w')  
        
        for i in list:  
            name=total_xml[i][:-4]+'\n'  
            if i in trainval:  
                ftrainval.write(name)  
                if i in train:  
                    ftrain.write(name)  
                else:  
                    fval.write(name)  
            else:  
                ftest.write(name)  
        
        ftrainval.close()  
        ftrain.close()  
        fval.close()  
        ftest.close()
        print("Generate txt in ImageSets done.")

    if annotation_mode == 0 or annotation_mode == 2:
        print("Generate 2007_train.txt and 2007_val.txt for train.")
        for year, image_set in VOCdevkit_sets:
            image_ids = open(os.path.join(VOCdevkit_path, 'VOC%s/ImageSets/Main/%s.txt'%(year, image_set)), encoding='utf-8').read().strip().split()
            list_file = open('%s_%s.txt'%(year, image_set), 'w', encoding='utf-8')
            for image_id in image_ids:
                list_file.write('%s/VOC%s/JPEGImages/%s.jpg'%(os.path.abspath(VOCdevkit_path), year, image_id))

                convert_annotation(year, image_id, list_file)
                list_file.write('\n')
            list_file.close()
        print("Generate 2007_train.txt and 2007_val.txt for train done.")
"""

if __name__ == "__main__":
    imagesets = ["train","test"]#训练/测试数据
    random.seed(0)
    if annotation_mode == 0 or annotation_mode == 1:
        for names in image_classes:
            print("Generate txt in ImageSets.")
            xmlfilepath     = os.path.join(VOCdevkit_path, 'data_bird_box/annotations/train/%s/'%(names))
            saveBasePath    = os.path.join(VOCdevkit_path, 'data_bird_box/ImageSets/Main/%s'%(names))
            temp_xml        = os.listdir(xmlfilepath)
            total_xml       = []
            for xml in temp_xml:
                if xml.endswith(".xml"):
                    total_xml.append(xml)

            num     = len(total_xml)  
            list    = range(num)  
            tv      = int(num*trainval_percent)  
            tr      = int(tv*train_percent)  
            trainval= random.sample(list,tv)  
            train   = random.sample(trainval,tr)  
        
            print("train and val size",tv)
            print("train size",tr)
            ftrainval   = open(os.path.join(saveBasePath,'trainval.txt'), 'w')  
            ftest       = open(os.path.join(saveBasePath,'test.txt'), 'w')  #ftest用test中的数据
            ftrain      = open(os.path.join(saveBasePath,'train.txt'), 'w')  
            fval        = open(os.path.join(saveBasePath,'val.txt'), 'w')  
        
            for i in list:  
                name=total_xml[i][:-4]+'\n'  
                if i in trainval:  
                    ftrainval.write(name)  
                    if i in train:  
                        ftrain.write(name)  
                    else:  
                        fval.write(name)  
                else:  
                    ftest.write(name)  
        
        ftrainval.close()  
        ftrain.close()  
        fval.close()  
        ftest.close()
        print("Generate txt in ImageSets done.")

    if annotation_mode == 0 or annotation_mode == 2:
        print("Generate train.txt and val.txt for train.")
        for imageset in imagesets:
            list_file = open('%s.txt'%(imageset), 'w', encoding='utf-8')
            for names in image_classes:
                image_ids = open(os.path.join(VOCdevkit_path, 'data_bird_box/ImageSets/Main/%s/%s.txt'%(names,imageset)), encoding='utf-8').read().strip().split()
                for image_id in image_ids:
                    list_file.write('%s/data_bird_box/JPGimages/%s/%s/%s.jpg'%(os.path.abspath(VOCdevkit_path),imageset,names,image_id))

                    convert_annotation(names, image_id, list_file)
                    list_file.write('\n')
            list_file.close()
            print("Generate %s for train done."%imageset)
        print("Generate train.txt and val.txt for train done.")
    

Generate txt in ImageSets.
train and val size 54
train size 48
Generate txt in ImageSets.
train and val size 54
train size 48
Generate txt in ImageSets.
train and val size 52
train size 46
Generate txt in ImageSets.
train and val size 54
train size 48
Generate txt in ImageSets.
train and val size 39
train size 35
Generate txt in ImageSets.
train and val size 36
train size 32
Generate txt in ImageSets.
train and val size 47
train size 42
Generate txt in ImageSets.
train and val size 43
train size 38
Generate txt in ImageSets.
train and val size 53
train size 47
Generate txt in ImageSets.
train and val size 54
train size 48
Generate txt in ImageSets done.
Generate train.txt and val.txt for train.
Generate train for train done.
Generate test for train done.
Generate train.txt and val.txt for train done.


# 调试

In [4]:
import os
import random
import xml.etree.ElementTree as ET

from utils.utils import get_classes

In [13]:
VOCdevkit_path  = 'E:/university/2022 semester'
image_classes = "001.Black_footed_Albatross"
image_id = "Black_Footed_Albatross_0001_796111"
in_file = open(os.path.join(VOCdevkit_path , 'data_bird_box\\annotations\\train\\%s\\%s.xml'%(image_classes,image_id)), encoding='utf-8')
tree=ET.parse(in_file)
root = tree.getroot()
for obj in root.iter('object'):
        difficult = 0 
        if obj.find('difficult')!=None:
            difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(float(xmlbox.find('xmin').text)), int(float(xmlbox.find('ymin').text)), int(float(xmlbox.find('xmax').text)), int(float(xmlbox.find('ymax').text)))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))

<Element 'annotation' at 0x0000000008D93EF8>


In [27]:
import random
print(range(100))
random_num = random.sample(range(100),100)
print(len(random_num))

range(0, 100)
100


In [35]:
for names in image_classes:
    print("Generate txt in ImageSets.")
    print(names)
    xmlfilepath     = os.path.join(VOCdevkit_path, 'data_bird_box/annotations/%s/%s/'%(imageset,names))
    saveBasePath    = os.path.join(VOCdevkit_path, 'data_bird_box/ImageSets/Main')
    temp_xml        = os.listdir(xmlfilepath)
    print(temp_xml)

Generate txt in ImageSets.
001.Black_footed_Albatross
['Black_Footed_Albatross_0001_796111.xml', 'Black_Footed_Albatross_0002_55.xml', 'Black_Footed_Albatross_0005_796090.xml', 'Black_Footed_Albatross_0007_796138.xml', 'Black_Footed_Albatross_0008_796083.xml', 'Black_Footed_Albatross_0009_34.xml', 'Black_Footed_Albatross_0010_796097.xml', 'Black_Footed_Albatross_0016_796067.xml', 'Black_Footed_Albatross_0017_796098.xml', 'Black_Footed_Albatross_0019_796104.xml', 'Black_Footed_Albatross_0023_796059.xml', 'Black_Footed_Albatross_0026_796095.xml', 'Black_Footed_Albatross_0031_100.xml', 'Black_Footed_Albatross_0035_796140.xml', 'Black_Footed_Albatross_0037_796120.xml', 'Black_Footed_Albatross_0038_212.xml', 'Black_Footed_Albatross_0039_796132.xml', 'Black_Footed_Albatross_0040_796066.xml', 'Black_Footed_Albatross_0041_796108.xml', 'Black_Footed_Albatross_0042_796071.xml', 'Black_Footed_Albatross_0045_796129.xml', 'Black_Footed_Albatross_0046_18.xml', 'Black_Footed_Albatross_0047_796064.xml